In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [3]:
importedResponse=pd.read_pickle('data/ZipcodeMSPPS.pkl')
response=importedResponse[['RegionName','City','State','2000_agg','2010_agg','pct_delta']]
response=response.rename(columns={'RegionName': 'Zip_Code'})

In [4]:
response.head(10)

,Zip_Code,City,State,2000_agg,2010_agg,pct_delta
1,60629,Chicago,IL,98.682400,115.605500,1.171491
2,90650,Norwalk,CA,130.505400,229.691000,1.760011
7,90280,South Gate,CA,130.197000,212.896167,1.635185
10,60618,Chicago,IL,151.892400,265.168433,1.745765
13,91331,Los Angeles,CA,104.836333,195.163200,1.861599
14,8701,Lakewood Township,NJ,74.834500,135.057367,1.804747
15,92335,Fontana,CA,79.024267,118.977400,1.505581
21,90805,Long Beach,CA,123.739133,222.963300,1.801882
24,94565,Pittsburg,CA,117.949467,120.324433,1.020135
27,92336,Fontana,CA,84.988300,127.604600,1.501437


In [5]:
cdd=pd.read_pickle('data/cleaned_demographic_features.pkl')
cdd=cdd.rename(columns={'GEO.id2' : 'Zip_Code'})
print cdd.shape
cdd.head(10)

(33120, 59)


,GEO.id,Zip_Code,GEO.display-label,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056
1,8600000US00601,00601,ZCTA5 00601,18570,12.8,16.2,13.9,11.7,13.3,13.1,10.2,5.7,3.1,48.9,6.6,8.5,6.9,5.4,6.3,6.3,4.9,2.7,1.3,51.1,6.3,7.7,7,6.3,6.9,6.8,5.3,2.9,1.8,93.1,3.1,0.4,0,0,99.5,77.8,5.1,2.6,20.3,9.6,20.3,10.2,3.3,10.1,5.5,39.5,29,84.3,15.7,2.9,0.5,2.84,3.27,7744,2.87
2,8600000US00602,00602,ZCTA5 00602,41520,11.5,15.2,13.5,12.9,14.6,14,10.1,5.7,2.3,49.1,5.9,7.7,6.7,6.4,7,6.8,4.9,2.6,1.1,50.9,5.6,7.5,6.8,6.5,7.6,7.2,5.2,3.1,1.3,86.7,5.3,0.3,0.1,0,99.4,77.6,5.1,2.1,17.4,8,20.5,10.5,3,10,4.9,37.6,25.9,83,17,2.6,1.3,2.76,3.20,18073,2.62
3,8600000US00603,00603,ZCTA5 00603,54689,12.2,14.2,13,13,12.7,12.6,12,7,3.3,48.6,6.4,7.3,6.5,6.4,6,5.8,5.6,3.2,1.4,51.4,5.8,7,6.5,6.6,6.6,6.7,6.4,3.7,2.1,82.9,7.6,0.3,0.2,0,98.5,71.1,4.7,1.7,22,10.9,26.4,12.7,4.1,13.8,6.8,34,31.1,82.5,17.5,3.6,1.5,2.53,3.06,25653,2.51
4,8600000US00606,00606,ZCTA5 00606,6615,12.8,15.1,14.7,12.1,13.1,13.4,10.5,5.5,2.9,49.4,6.6,7.5,7.1,6.1,6.5,6.3,5.2,2.6,1.4,50.6,6.2,7.5,7.6,6,6.6,7,5.4,2.9,1.5,88.9,4.7,0.4,0,0.1,99.4,74.6,6.2,2.7,18.4,8.1,22.8,12.7,4.5,10.1,4.7,37.5,28.4,83.6,16.4,1.7,0.8,2.75,3.24,2877,2.83
5,8600000US00610,00610,ZCTA5 00610,29016,11.8,15.1,12.6,13.4,13.9,12.6,11.2,6.7,2.8,48.6,6.2,7.7,6.4,6.6,6.5,5.9,5.1,3.2,1.1,51.4,5.6,7.5,6.2,6.9,7.4,6.7,6.1,3.5,1.7,82,7.2,0.3,0.1,0,99.2,74.4,4.9,1.9,19.9,9.4,22.8,11,3.4,11.9,5.8,36.5,29.4,85.9,14.1,1.9,0.8,2.67,3.14,12618,2.66
6,8600000US00612,00612,ZCTA5 00612,67010,11.4,14.3,12.5,12.6,13.1,12.2,11.8,7.7,4.2,47.4,6,7.3,6.2,6,6.2,5.5,5.2,3.4,1.7,52.6,5.5,7,6.4,6.7,6.9,6.6,6.7,4.4,2.5,84.3,6.2,0.4,0.1,0,99.1,71.7,4.8,1.8,22.4,10.2,25.6,10.7,3.8,14.8,7.8,33.8,33.7,84,16,3.1,2.2,2.55,3.05,30992,2.54
7,8600000US00616,00616,ZCTA5 00616,11017,11.8,14.9,11.8,13.3,14.5,11.4,11.9,6.6,3.7,48.3,6.1,7.5,5.9,6.1,6.9,5.2,5.5,3.2,1.6,51.7,5.7,7.3,5.9,7.2,7.6,6.2,6.4,3.4,2.1,84.8,7.1,0.3,0,0,99.5,73.3,4.4,1.5,20.5,9.1,24.3,12,5,12.2,6.2,36,31.6,84.6,15.4,2.3,1,2.65,3.16,4896,2.53
8,8600000US00617,00617,ZCTA5 00617,24597,13.4,15.1,13,15,12.7,10.9,10.6,6.1,3.2,48.1,6.8,7.8,6.3,7,5.9,5.1,5,2.8,1.3,51.9,6.5,7.4,6.7,8,6.9,5.7,5.5,3.3,1.8,80.6,7.7,0.3,0.1,0,99.4,74.7,5.2,2.2,23.6,11.5,22.8,11.4,3.9,11.4,5.5,38.9,29,85.7,14.3,1.9,2.5,2.71,3.18,10594,2.80
9,8600000US00622,00622,ZCTA5 00622,7853,9.6,13,11,11.1,12.5,13.1,14.6,9.7,5.3,48.2,4.9,6.5,5.2,5.3,6.4,6.1,7.1,4.6,2.1,51.8,4.7,6.5,5.8,5.8,6.2,7.1,7.6,5,3.2,85.8,5,0.2,0,0,98.2,69.1,5.5,2.6,15.5,5.9,26.9,13.4,4.2,13.4,7.3,28.9,37.8,36.6,63.4,3.7,4.9,2.45,2.97,8714,2.51
10,8600000US00623,00623,ZCTA5 00623,43061,12.3,14.4,11.3,14.1,13.3,11.3,11.7,7.5,4,47.8,6.4,7.3,5.6,6.6,6.4,5.2,5.4,3.5,1.6,52.2,6.1,7.1,5.7,7.6,6.9,6.1,6.3,4,2.3,83.8,5.5,0.3,0.1,0,99.1,71.6,4.9,2.2,19.3,9.4,25,11.5,4,13.5,7.3,35.7,32.6,77.6,22.4,1.8,2.7,2.58,3.08,21426,2.61


In [6]:
merged=pd.merge(response, cdd, on=['Zip_Code'])
merged.head(10)

,Zip_Code,City,State,2000_agg,2010_agg,pct_delta,GEO.id,GEO.display-label,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056
0,60629,Chicago,IL,98.682400,115.605500,1.171491,8600000US60629,ZCTA5 60629,113916,18.1,17.7,16,15.4,12.5,10.2,5.5,2.6,1.8,49.4,9.2,8.9,8.1,7.8,6.2,4.9,2.6,1.1,0.7,50.6,8.9,8.9,8,7.6,6.3,5.3,3,1.5,1.2,41,23,0.7,0.5,0,67.2,78.1,8.7,4.1,22.6,12,18.1,8.7,2.3,9.4,4.1,53.6,20.1,88.6,11.4,4.3,1.7,3.77,4.24,34000,3.21
1,90650,Norwalk,CA,130.505400,229.691000,1.760011,8600000US90650,ZCTA5 90650,105549,14.2,16.8,15.2,14.3,13.6,11.7,7.1,4.1,2.7,49.6,7.4,8.8,7.9,7.1,6.8,5.6,3.3,1.7,1,50.4,6.9,8,7.3,7.2,6.8,6.1,3.9,2.4,1.7,49.4,4.4,1.1,12,0.4,70.1,83.2,8.7,3.6,18.6,7.8,12.6,5.3,1.7,7.3,4.3,50.4,28,96.6,3.4,1.3,0.9,3.83,4.10,28083,3.57
2,90280,South Gate,CA,130.197000,212.896167,1.635185,8600000US90280,ZCTA5 90280,94396,16.7,18.1,16.2,14.9,13.1,10.4,6.2,2.9,1.6,49.1,8.5,9,8.1,7.4,6.6,4.8,2.8,1.2,0.6,50.9,8.2,9.1,8.1,7.5,6.5,5.5,3.3,1.7,1,50.5,0.9,0.9,0.8,0.1,94.8,86.6,9.7,5,20.2,10.7,9.8,4.5,1.3,5.4,3,59.3,21.9,96.3,3.7,1.9,0.7,4.05,4.24,24160,3.78
3,60618,Chicago,IL,151.892400,265.168433,1.745765,8600000US60618,ZCTA5 60618,92084,14.5,10.4,19,20.7,13.8,10.2,6.3,3.3,1.9,50.1,7.2,5.3,9.5,10.7,7.3,5.1,2.9,1.3,0.7,49.9,7.2,5.1,9.4,10,6.5,5.1,3.4,2,1.2,65.8,3.5,0.7,5.3,0,46.4,57.5,6.2,2.4,12.3,6.2,29.2,14,1.8,15.2,4.1,31.7,15.9,89.5,10.5,4,1.3,2.59,3.34,39547,2.49
4,91331,Los Angeles,CA,104.836333,195.163200,1.861599,8600000US91331,ZCTA5 91331,103689,16.5,17.7,16.6,14.8,13.3,10.3,5.9,3.2,2,50.5,8.4,9.1,8.7,7.7,6.8,5,2.7,1.3,0.7,49.5,8.1,8.7,7.9,7.1,6.5,5.3,3.2,1.9,1.2,44.4,3.3,0.9,4.4,0.1,87.8,86.4,10.4,4.8,19.9,9.5,9.8,4.5,1.4,5.3,3.2,60.1,26,94.5,5.5,1.3,1.2,4.60,4.63,23780,4.23
5,92335,Fontana,CA,79.024267,118.977400,1.505581,8600000US92335,ZCTA5 92335,95397,18.7,19.6,16.5,13.8,12.9,9.4,4.8,2.6,1.4,50,9.6,10,8.6,6.8,6.4,4.7,2.3,1.1,0.5,50,9.3,9.5,8.1,7.1,6.4,4.7,2.6,1.5,0.9,48.6,4.8,1.2,1.6,0.4,81.7,83.7,9.9,5.2,19.2,11.2,12.3,5.7,1.6,6.6,3.7,60.7,20.1,93.9,6.1,3.1,1.3,4.18,4.42,24246,3.92
6,90805,Long Beach,CA,123.739133,222.963300,1.801882,8600000US90805,ZCTA5 90805,93524,17.1,18.1,16.4,14.1,13.4,10.7,6.1,2.7,1.5,48.4,8.7,9.1,7.9,6.7,6.5,5.1,2.8,1.1,0.5,51.6,8.3,9,8.4,7.4,7,5.7,3.2,1.6,1,31.2,20.7,0.8,10.7,2.5,55.8,77.4,9,4.4,24.5,14.4,17.5,8.2,1.8,9.3,3.4,53,18.6,94.2,5.8,3.4,1,3.56,4.00,27646,3.33
7,94565,Pittsburg,CA,117.949467,120.324433,1.020135,8600000US94565,ZCTA5 94565,84641,16,15.4,15.9,14.3,13.5,12.5,7,3.4,1.9,49,8.1,7.8,8,7.1,6.6,6,3.3,1.5,0.7,51,7.9,7.5,7.9,7.3,7,6.6,3.8,1.9,1.2,37.8,16.1,0.9,14.2,0.9,45.6,76.5,7.6,3.6,18.8,10,17.4,7.4,1.6,9.9,3.7,46.5,20.3,92.3,7.7,2.9,2,3.27,3.68,27897,3.34
8,92336,Fontana,CA,84.988300,127.604600,1.501437,8600000US92336,ZCTA5 92336,88419,16.4,19.3,14.1,15.4,15.3,11,5.5,2.1,0.9,49.5,8.5,9.9,7,7.4,7.5,5.4,2.7,0.9,0.3,50.5,8,9.4,7,8,7.7,5.6,2.8,1.2,0.6,46.3,13.2,0.9,10.6,0.2,55.2,89.3,7.2,3.5,14,7.5,7,3.5,0.5,3.5,1.1,60.5,15.7,95.5,4.5,1,2,3.97,4.10,23271,4.30
9,92503,Riverside,CA,92.957033,128.744833,1.384993,8600000US92503,ZCTA5 92503,84519,16.3,17.4,15.1,13.7,14,11.3,6.8,3.3,1.9,49.8,8.4,9,7.8,6.8,7.1,5.5,3.3,1.4,0.7,50.2,7.9,8.5,7.3,7,6.9,5.8,3.6,1.9,1.2,54.8,4.9,1.2,6.3,0.4,57.7,79.6,7.3,3.6,14.9,7.5,15.5,6.9,1.7,8.6,4.3,49.9,21.4,94.6,5.4,2,1.5,3.57,3.93,24754,3.55


In [9]:
importedResponse.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,...,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2000_agg,2010_agg,pct_delta
1,84630,60629,Chicago,IL,Chicago,Cook,2,83.37290,81.9039,86.2227,88.2281,92.4941,88.4356,84.2880,86.4587,87.2849,82.1885,84.3404,85.3516,83.9546,80.3261,80.8496,85.4139,87.8756,86.3272,86.9252,88.9441,84.8332,82.1182,82.8501,85.955400,85.4117,85.6305,90.6093,92.3588,95.8791,95.4600,97.4595,94.5612,91.3699,91.5263,90.7089,94.4271,92.9856,93.5025,95.1732,98.5480,97.9914,100.6991,101.1288,...,95.8577,101.924600,106.3171,109.1850,115.7075,120.956500,124.7645,123.4396,118.7330,122.6573,106.9069,103.3210,102.9773,101.9681,108.3858,102.5008,111.5418,110.0932,103.3630,94.8442,99.4525,103.1513,123.4814,119.6632,125.0165,118.4917,112.1278,121.5239,120.1162,115.293300,103.5425,105.5923,121.6394,126.17420,125.6553,116.9917,113.141500,118.9884,107.6563,107.276200,109.5701,116.6246,111.9092,114.7804,127.9313,132.5965,131.306100,98.682400,115.605500,1.171491
2,96193,90650,Norwalk,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,3,117.39690,118.0225,113.3146,112.0347,114.1521,113.2268,111.2448,109.2356,112.6931,115.6738,115.9940,108.9467,113.3924,110.7738,108.5902,110.6241,113.3821,117.6624,114.2688,112.9188,114.2251,116.3165,111.2844,111.104200,112.2943,117.9401,117.9640,117.6704,117.3105,114.7438,115.1906,113.4917,119.0176,118.3696,119.5861,120.8136,122.6741,122.6501,123.3720,124.8398,125.4529,126.7413,127.6706,...,219.8136,236.888700,234.3855,225.6425,229.9889,224.835300,227.7036,234.5118,234.7218,228.2131,221.8379,231.7794,230.8116,249.9534,258.6240,247.0731,248.9392,250.8335,266.6558,266.6535,268.9651,282.5015,287.1165,293.1764,281.2928,291.0114,282.9296,298.6052,289.6318,294.915300,300.2414,303.2769,291.9878,303.57810,314.2814,327.2802,317.249600,312.0753,297.1876,287.113100,305.2852,314.4935,339.6102,341.7029,339.3655,336.4867,343.498500,130.505400,229.691000,1.760011
7,96125,90280,South Gate,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,8,118.30150,121.3851,117.8144,122.3694,124.6468,122.9418,119.5063,118.1009,122.6550,121.1318,118.7361,112.8810,115.2853,114.9234,116.5601,115.7037,115.4531,117.2410,123.1254,117.5763,116.3737,117.3234,112.4569,116.410400,120.8716,121.4901,117.2832,119.2311,118.1527,123.1749,115.0169,114.8405,117.6092,119.9959,121.5556,118.0527,118.2184,119.1929,122.5280,119.8428,115.2839,117.3491,119.3010,...,NaN,214.758811,202.8838,NaN,NaN,191.730827,NaN,202.3067,213.5910,NaN,201.5256,NaN,228.0380,NaN,209.3866,207.7554,222.7143,214.7797,245.8332,237.6460,215.1479,236.1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.610169,NaN,NaN,NaN,272.65861,NaN,NaN,242.248062,NaN,NaN,266.089109,267.5440,281.4984,303.1221,312.5424,NaN,NaN,279.620853,130.197000,212.896167,1.635185
10,84620,60618,Chicago,IL,Chicago,Cook,11,130.28169,121.4685,127.4808,125.9311,119.8565,107.3306,103.7671,112.6521,118.0756,107.7247,NaN,102.1097,107.0049,110.6245,122.6887,115.8170,110.1592,118.9129,119.0890,114.2983,107.8967,NaN,NaN,138.190955,124.6543,122.2254,143.4391,127.7708,118.8031,125.8294,133.5715,123.8324,127.8238,120.7668,118.3218,118.3846,133.5493,152.1399,134.9239,140.7404,167.3603,173.1714,152.8913,...,224.3826,231.606600,NaN,222.1806,202.4930,219.852200,223.6268,237.7173,249.2751,226.0825,224.8802,238.2890,236.9822,215.7680,196.7915,NaN,226.3855,251.

In [8]:
response[response['Zip_Code']=='08701']

,Zip_Code,City,State,2000_agg,2010_agg,pct_delta
